Based on code: http://lethain.com/genetic-algorithms-cool-name-damn-simple/

Function to solve: http://www.geatbx.com/docu/fcnindex-01.html#P129_5426

In [2]:
from random import randint, random
from operator import add
from functools import reduce
from numpy.random import choice
import string
import numpy as np
import math

#### Create member of population and population 

In [3]:
def individual(min, max):
     return [ randint(min,max) for x in range(2)] #range(2) for 2 parameters

In [4]:
def population(count, min, max):
    return[individual(min, max) for x in range(count)]

#### Fitness function 

Searching for local minimum in space [-2.047;2.047]

In [6]:
def fitness(individual): #fitness function
    fx=(100*pow((pow(individual[0]*0.001,2))-individual[1]*0.001,2)
    +pow((1-individual[0]*0.001),2))
    return fx 

#### Population parameters 

In [9]:
def grade(pop): #average fitness for each individual
    summed= reduce(add, (fitness(x) for x in pop),0)
    return(summed/(len(pop)*1.0))

#### new population of parents - roulette selection

In [10]:
def roulette_selection(lista):
    fitness_values = [ (fitness(x),x) for x in lista] #fit function and individual
    
    def rank(value):
        return (1/math.sqrt(value)) # we will use this rank function in roulette to reduce hudge distproportions 
                                    # between fitness values for different individuals. This leads to bigger 
                                    # variation of individuals in the next population
    
    fitness_sum = sum(rank(fit) for fit,x in fitness_values)
    weights =[rank(fit)/fitness_sum for fit,x in fitness_values]
    parents_length=len(weights)
    parents_indicator = choice(range(parents_length),int(parents_length),p=weights) 
    selected_parents=[fitness_values[i][1] for i in parents_indicator]
    return (selected_parents)

In [11]:
def convert_to_binary(data, bit_length):
    return [(format(x,bit_length),format(y,bit_length)) for x,y in data]

In [12]:
def convert_to_integer(child):
    return [int(child[0],2),int(child[1],2)]

## Breeding & mutation rules 

In [14]:
def evolve(pop, retain=0.2, mutate=0.1):
    
    graded=sorted(pop, key=fitness)
    retain_length = int(len(graded)*retain) #select individuals with the best fitness to stay unchanged
    
    ### retain some parents
    retained_parents=[]
    retained_parents = graded[:retain_length] 
    not_retained_parents = graded[retain_length:]
    
    new_parents=roulette_selection(not_retained_parents)
    new_parents_binary=convert_to_binary(new_parents,bit_length="014b")
    
    ### assig create split point (to binary crossover) for each parent
    n_of_bits=len(new_parents_binary[0][0])
    length_new_parents=len(new_parents)
    split_point=choice(n_of_bits-4,size=int(length_new_parents/2),replace=True)+2 

    # if we have odd number of new parents, take middle value and append to parents_to retain
    if (int(length_new_parents/2))%1!=0:
        to_append=new_parents_binary[int(length_new_parents/2)+1]
        to_append=[int(to_append[0],2),int(to_append[1],2)]
        retained_parents.append(to_append)
        
    # take the rest of new parents to crossover
    for i in range(int(length_new_parents/2)):
        parent1,parent2=new_parents_binary[i-1],new_parents_binary[length_new_parents-i-1] #take first and last parent
        child1=(parent1[0][:split_point[i]]+parent2[0][split_point[i]:],parent1[1][:split_point[i]]+parent2[1][split_point[i]:])
        child2=(parent2[0][:split_point[i]]+parent1[0][split_point[i]:],parent2[1][:split_point[i]]+parent1[1][split_point[i]:])
        
        # mutate with proabbility="mutate"
        if choice([0,1], 1, p=[1-mutate,mutate])==1:            
            index1=randint(3,n_of_bits-1)
            child1=(child1[0][:index1] + str("1") + child1[0][index1 + 1:],child1[1]) # change only 1st. element of 1st. child
        if choice([0,1], 1, p=[1-mutate,mutate])==1:
            index2=randint(3,n_of_bits-1)      
            child2=(child2[0],child2[1][:index2] + str("1") + child2[1][index2 + 1:])# change only 2st. element of 2st. child
        
        #change to integer
        int_child1=convert_to_integer(child1)
        int_child2=convert_to_integer(child2)      
        retained_parents.append(int_child1)
        retained_parents.append(int_child2)
    return (retained_parents)

## Run algorithm to find minimum fitness value


In [30]:
n_of_individuals = 100 #number of individuals in population
i_min = -2047 # min value of individual
i_max = 2047  # max value of individual
n_of_iteretions = 30
p = population(n_of_individuals, i_min, i_max)
fitness_history = [grade(p),]
for i in range(n_of_iteretions):
    populacja = p
    p = evolve(p)
    fitness_history.append(grade(p))
    
for avg_fitness in fitness_history:
    print (avg_fitness)
print("\nMinimum found value: " ,fitness(p[0]))
print("\nIndividual of minimum value: " ,p[0])

567.0294152199912
351.29694049839605
266.72014377383397
208.507670105486
131.56894369997997
116.114915041199
52.848899260728
48.89024155834
37.070528656501004
33.79844594548099
20.135768775313
16.821007530966995
23.492801684151996
31.627546321295995
13.641337201808994
18.546115167520995
8.827443786813998
6.257342803646
8.053826349056997
3.7543668617470014
11.441389926542998
13.444912033169997
4.155568656702
4.75248770665
1.3510228918210005
2.257861841259001
1.4877595329000004
2.660318980008999
5.342660315085001
2.1005439751339994
1.7275543867569991

Minimum found value:  0.005140999999999994

Individual of minimum value:  [990, 973]


#### Actual minimum at point: [1,1], minimum value = 0 

More: http://www.geatbx.com/docu/fcnindex-01.html#P129_5426